In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

In [15]:
import sys
#print(torch.version.cuda)
print(torch.version.cuda)
#查看torch版本
print(torch.__version__)
# 查看python版本
print(sys.version)

11.1
1.8.0+cu111
3.6.15 (default, Dec  3 2021, 18:25:24) [MSC v.1916 64 bit (AMD64)]


In [17]:
# 简单的图卷积网络（GCN）模型
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)  # 第一个图卷积层
        x = torch.relu(x)  # 激活函数
        x = self.conv2(x, edge_index)  # 第二个图卷积层
        return x

In [18]:
nodes = torch.tensor([[1.0], [2.0], [1.5], [2.5]], dtype=torch.float)  # 每个路口的流量特征（示例）

# 连接矩阵，表示图的边（边：路口连接关系）
# 假设路口0和1相连，路口1和2相连，路口2和3相连，路口3和0相连
edges = torch.tensor([[0, 1], [1, 2], [2, 3], [3, 0], [0, 2], [1, 3]], dtype=torch.long).t().contiguous()


In [19]:
# 数据封装成PyG（PyTorch Geometric）所需的Data对象
data = Data(x=nodes, edge_index=edges)

# 创建一个GCN模型
model = GCN(in_channels=1, hidden_channels=8, out_channels=1)

In [26]:
print(data.x)
print(data.edge_index)
print(data.edge_index.shape)
print(edges)

tensor([[1.0000],
        [2.0000],
        [1.5000],
        [2.5000]])
tensor([[0, 1, 2, 3, 0, 1],
        [1, 2, 3, 0, 2, 3]])
torch.Size([2, 6])
tensor([[0, 1, 2, 3, 0, 1],
        [1, 2, 3, 0, 2, 3]])


In [20]:
# 损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [21]:
target = torch.tensor([[1.1], [2.1], [1.6], [2.6]], dtype=torch.float)  # 目标流量数据（假设）


In [23]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    # 前向传播
    optimizer.zero_grad()
    out = model(data)  # 模型预测的流量
    loss = criterion(out, target)  # 计算损失

    # 反向传播
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# 测试模型（预测结果）
model.eval()
predictions = model(data)
print("Predicted traffic flow for each intersection:", predictions.detach().numpy())

Epoch 1/100, Loss: 0.7878574132919312
Epoch 11/100, Loss: 0.27098900079727173
Epoch 21/100, Loss: 0.30905023217201233
Epoch 31/100, Loss: 0.2708536684513092
Epoch 41/100, Loss: 0.27389979362487793
Epoch 51/100, Loss: 0.2707606852054596
Epoch 61/100, Loss: 0.26964837312698364
Epoch 71/100, Loss: 0.2697254419326782
Epoch 81/100, Loss: 0.26942306756973267
Epoch 91/100, Loss: 0.26924368739128113
Predicted traffic flow for each intersection: [[1.756579 ]
 [1.6616933]
 [1.9633197]
 [2.033201 ]]
